<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/Reranking_Result_in_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hướng dẫn về các biện pháp nâng cao hiệu quả cho RAG (post-retrieval)**

Đây là notebook cho các bạn để có thể tìm hiểu về cách nâng cao hiệu quả cho RAG pipeline, với LlamaIndex và open-source LLM như Llama 3. Phương pháp thực hiện ở đây được triển khai sau khi tìm tài liệu (retrieval).

***Cài đặt các thư viện cần thiết:***

Trước tiên, việc cần làm đầu tiên là cài đặt các thư viện cần thiết cho RAG pipeline. Điều này chúng ta có thể thực hiện tương tự như notebook trước; điểm khác biệt ở đây là chúng ta cài thêm ColBERT để sắp xếp lại kết quả sau khi đã tìm tài liệu xong.

ColBERT là một phiên bản của BERT (Bidirectional Encoder Representations from Transformers), được fine-tune để truy lục thông tin (retrieval) dựa trên bối cảnh. Với thư viện MaxSim (scalable vector-similarity), ColBERT có thể tìm dữ liệu nhanh hơn BERT nhiều lần, và có thể scale với tập dữ liệu lớn. Điều này khiến ColBERT trở thành lựa chọn chính cho việc rerank thông tin.

Các bạn có thể tham khảo thêm về ColBERT từ paper gốc này: [ColBERT: Eicient and Eective Passage Search via
Contextualized Late Interaction over BERT](https://arxiv.org/pdf/2004.12832)

Đồng thời, codebase của ColBERT đã được công khai trên GitHub: [ColBERT GitHub](https://github.com/stanford-futuredata/ColBERT)

In [ ]:
%%writefile requirements.txt
llama-index
llama-index-llms-huggingface
llama-index-embeddings-huggingface
Unstructured[md]
einops
accelerate
sentence-transformers
bitsandbytes
llama-index-postprocessor-colbert-rerank

Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/qdrant/qdrant_client.git to /tmp/pip-req-build-jd2jikho
  Running command git clone --filter=blob:none --quiet https://github.com/qdrant/qdrant_client.git /tmp/pip-req-build-jd2jikho
  Resolved https://github.com/qdrant/qdrant_client.git to commit f63009f00bf1f1e415b30763898ba7146f0eaece
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install pypdf

Chúng ta sẽ cho notebook truy cập vào HuggingFace tương tự như ví dụ trước (bằng secret keys).

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

Về dữ liệu cần dùng, chúng ta sẽ sử dụng form 10-K của Alphabet, tương tự như trong notebook trước.

In [ ]:
import urllib.request

url = "https://abc.xyz/assets/43/22/5deefff4fbec54014ae97b340c22/34ac6dab5f586b2e6e008b99fe683e35.pdf"

urllib.request.urlretrieve(url, "Alphabet2023Form10-K.pdf")


('Alphabet2023Form10-K.pdf', <http.client.HTTPMessage at 0x7a13afa38af0>)

***Indexing và lưu trữ tài liệu:***

Tiếp đó chúng ta sẽ nhập các module chính của LlamaIndex (bao gồm vector DB chính) vào pipeline, tương tự như mục 1. Điểm khác biệt chính của chúng ta là chúng ta sẽ thêm một mục (colbert_rerank) để sắp xếp lại dữ liệu.

Các bạn có thể thấy là LlamaIndex có hỗ trợ thêm reranker này trong library gốc.

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.postprocessor.colbert_rerank import ColbertRerank

reader = SimpleDirectoryReader(input_files=["/content/Alphabet2023Form10-K.pdf"])
data = reader.load_data()

Về system prompt và embedding model, ví dụ này chúng ta sẽ sử dụng tương tự như trong các ví dụ trước (để đảm bảo Llama-3 trả lời bằng tiếng Việt).

In [ ]:
from llama_index.core import PromptTemplate
system_prompt = "Bạn là một trợ lý AI đắc lực. Hãy trả lời các câu hỏi càng chính xác càng tốt. Lưu ý là bạn chỉ trả lời bằng tiếng Việt và không dùng ngôn ngữ nào khác."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LLM và tokenizer chúng ta sẽ sử dụng Llama-3-8B-Instruct, tương tự như các ví dụ trước, với settings (chế độ cài đặt) tương tự.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=8192,
    max_new_tokens=512,
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": False,
        "temperature": 0.05,
        "top_p": 0.3,

    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from llama_index.core import Settings


Settings.embed_model = embed_model

Settings.chunk_size = 512
# Llama-3-8B-Instruct model
Settings.llm = llm

***Đặt câu hỏi cho LLM (querying):***

Ở phần này chúng ta sẽ đặt câu hỏi cho LLM. Điểm khác biệt chính ở đây là chúng ta cần tích hợp ColBERT vào trong pipeline.

Lưu ý là chúng ta cần phải đính kèm ColBERT với tư cách là model và tokenizer (suy cho cùng, đó vẫn là một LLM). Ở đây chúng ta sẽ chọn top 3 kết quả rank bởi ColBERT (*top_n=3* ).

In [ ]:
index = VectorStoreIndex.from_documents(data)

colbert_reranker = ColbertRerank(
    top_n=3,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Tiếp theo đó chúng ta sẽ đặt câu hỏi tương tự như với notebook trước (cho pre-retrieval). Lưu ý là ở mục này, colbert_reranker sẽ được coi là một bước để xử lý thông tin sau khi tìm bởi LLM (nên được nhập là node_postprocessors).

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3, streaming=True, node_postprocessors=[colbert_reranker],)



In [ ]:
response = query_engine.query("Tóm tắt thách thức với Alphabet Inc. trong năm 2023. "
 " Nêu từng thách thức với gạch đầu dòng, và dẫn nguồn theo từng trang cho mỗi thách thức.")

In [ ]:
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Alphabet Inc. trong năm 2023 phải đối mặt với các thách thức sau:

• **Tạm dừng hoạt động và giảm chi phí**: Alphabet Inc. sẽ phải tạm dừng hoạt động và giảm chi phí để giảm thiểu chi phí thuê văn phòng, với ước tính chi phí khoảng 0,5 tỷ USD trong quý đầu tiên của năm 2023. (Trang 32)

• **Cải thiện và tái định giá tài sản**: Alphabet Inc. sẽ phải cải thiện và tái định giá tài sản, bao gồm các máy chủ và thiết bị mạng, với ước tính giảm chi phí khấu hao khoảng 3,4 tỷ USD trong năm tài chính 2023. (Trang 31)

• **Tái định giá và tái tổ chức các hoạt động AI**: Alphabet Inc. sẽ phải tái định giá và tái tổ chức các hoạt động AI, bao gồm DeepMind, để phản ánh sự hợp tác ngày càng tăng giữa các hoạt động AI và các hoạt động khác của Alphabet. (Trang 31)

• **Tái định giá và tái tổ chức các báo cáo tài chính**: Alphabet Inc. sẽ phải tái định giá và tái tổ chức các báo cáo tài chính, bao gồm các báo cáo về các hoạt động AI, để phản ánh sự thay đổi trong các hoạt động của công ty. (Trang 45)

Chúng ta có thể kiểm tra kết quả của quá trình reranking, bằng cách in ID và nội dung của từng node một cùng với retrieval score. Ở trường hợp này thì Llama-3 chỉ tìm được 2 node cho nội dung chúng ta cần tìm.

In [ ]:
for node in response.source_nodes:
    print(node.id_)
    print(node.node.get_content())
    print("reranking score: ", node.score)
    print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("=====================================")

a553f72f-a898-4f8c-84ed-a15a0a2b8491
Table of Contents Alphabet Inc.
Note 9.
reranking score:  0.49754244089126587
retrieval score:  0.8370210160425409
f39beedb-9ab7-4c69-982c-8bf29a7ef90c
In addition, we are taking actions to optimize our global office space. As a result we expect to incur exit costs
relating to office space reductions of approximately $0.5 billion in the first quarter of 2023. We may incur
additional charges in the future as we further evaluate our real estate needs.
•In January 2023, we completed an assessment of the useful lives of our servers and network equipment,
resulting in a change in the estimated useful life of our servers and certain network equipment to six years,
which we expect to result in a reduction of depreciation of approximately $3.4 billion for the full fiscal year 2023
for assets in service as of December 31, 2022, recorded primarily in cost of revenues and R&D expenses.
•As AI is critical to delivering our mission of bringing our breakthrough i